In [170]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

import numpy as np 
import pandas as pd 

from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import word_tokenize
from sklearn.metrics.pairwise import cosine_distances


In [171]:
df = pd.read_csv('dataset/imdb_top_1000.csv')
df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469"
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411"
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444"
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000"
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000"


## Question
- Create new metadata with Genre, Overview, Director and Star 1-4
- Preprocessing the text with preprocessing part 1-3
- Create recommendation system with cosine distance
- Try the recommendation system is it good or bad ?

In [172]:
import warnings
warnings.filterwarnings('ignore')

In [173]:
def combine_metadata(row):
    return f"{row['Genre']} {row['Overview']} {row['Director']} {row['Star1']} {row['Star2']} {row['Star3']} {row['Star4']}"

In [174]:
df['Metadata'] = df.apply(combine_metadata, axis=1)
df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Metadata
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",Drama Two imprisoned men bond over a number of...
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411","Crime, Drama An organized crime dynasty's agin..."
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444","Action, Crime, Drama When the menace known as ..."
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000","Crime, Drama The early life and career of Vito..."
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000","Crime, Drama A jury holdout attempts to preven..."


In [175]:
metadata_row = df.loc[0, 'Metadata']
print(metadata_row)

Drama Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency. Frank Darabont Tim Robbins Morgan Freeman Bob Gunton William Sadler


## Preprocessing Part 1

1. Lowercase
1. Remove contraction
1. Remove number
1. Remove punctuation
1. Remove stopwords
1. Romove whitespace

### 1. Lowercase

In [176]:
def to_lower(text):
    return text.lower()

### 2. Remove contraction

In [177]:
contractions_dict = {    
    "ain't": "am not",
    "aren't": "are not",
    "can't": "cannot",
    "can't've": "cannot have",
    "'cause": "because",
    "could've": "could have",
    "couldn't": "could not",
    "couldn't've": "could not have",
    "didn't": "did not",
    "doesn't": "does not",
    "don't": "do not",
    "hadn't": "had not",
    "hadn't've": "had not have",
    "hasn't": "has not",
    "haven't": "have not",
    "he'd": "he had",
    "he'd've": "he would have",
    "he'll": "he will",
    "he'll've": "he will have",
    "he's": "he is",
    "how'd": "how did",
    "how'd'y": "how do you",
    "how'll": "how will",
    "how's": "how is",
    "i'd": "i had",
    "i'd've": "i would have",
    "i'll": "i will",
    "i'll've": "i will have",
    "i'm": "i am",
    "i've": "i have",
    "isn't": "is not",
    "it'd": "it had",
    "it'd've": "it would have",
    "it'll": "it will",
    "it'll've": "iit will have",
    "it's": "it is",
    "let's": "let us",
    "ma'am": "madam",
    "mayn't": "may not",
    "might've": "might have",
    "mightn't": "might not",
    "mightn't've": "might not have",
    "must've": "must have",
    "mustn't": "must not",
    "mustn't've": "must not have",
    "needn't": "need not",
    "needn't've": "need not have",
    "o'clock": "of the clock",
    "oughtn't": "ought not",
    "oughtn't've": "ought not have",
    "shan't": "shall not",
    "sha'n't": "shall not",
    "shan't've": "shall not have",
    "she'd": "she had",
    "she'd've": "she would have",
    "she'll": "she will",
    "she'll've": "she will have",
    "she's": "she is",
    "should've": "should have",
    "shouldn't": "should not",
    "shouldn't've": "should not have",
    "so've": "so have",
    "so's": "so is",
    "that'd": "that had",
    "that'd've": "that would have",
    "that's": "that is",
    "there'd": "there had",
    "there'd've": "there would have",
    "there's": "there is",
    "they'd": "they had",
    "they'd've": "they would have",
    "they'll": "they will",
    "they'll've": "they will have",
    "they're": "they are",
    "they've": "they have",
    "to've": "to have",
    "wasn't": "was not",
    "we'd": "we had",
    "we'd've": "we would have",
    "we'll": "we will",
    "we'll've": "we will have",
    "we're": "we are",
    "we've": "we have",
    "weren't": "were not",
    "what'll": "what will",
    "what'll've": "what will have",
    "what're": "what are",
    "what's": "what is",
    "what've": "what have",
    "when's": "when is",
    "when've": "when have",
    "where'd": "where did",
    "where's": "where is",
    "where've": "where have",
    "who'll": "who will",
    "who'll've": "who will have",
    "who's": "who is",
    "who've": "who have",
    "why's": "why is",
    "why've": "why have",
    "will've": "will have",
    "won't": "will not",
    "won't've": "will not have",
    "would've": "would have",
    "wouldn't": "would not",
    "wouldn't've": "would not have",
    "y'all": "you all",
    "y'all'd": "you all would",
    "y'all'd've": "you all would have",
    "y'all're": "you all are",
    "y'all've": "you all have",
    "you'd": "you had",
    "you'd've": "you would have",
    "you'll": "you will",
    "you'll've": "you will have",
    "you're": "you are",
    "you've": "you have"
}

In [178]:
def remove_contraction(text):

    list_kata = text.split()
    list_hasil = []

    for kata in list_kata:

        if kata in contractions_dict.keys():
            list_hasil.append(contractions_dict[kata])
        else:
            list_hasil.append(kata)

    hasil = ' '.join(list_hasil)
    return hasil 

### 3. Remove number

In [179]:
def remove_number(text):
    hasil = ''.join([char for char in text if not char.isnumeric()])
    return hasil

### 4. Remove punctuation

In [180]:
from string import punctuation

In [181]:
def remove_punctuation(text):
    hasil = ''.join([char for char in text if not char in punctuation])
    return hasil

### 5. Remove stopwords

In [182]:
# pip install nltk

In [183]:
import nltk
from nltk.corpus import stopwords

In [184]:
nltk.download('stopwords')
nltk.download('punkt') 

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\SWIFT\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\SWIFT\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [185]:
stop_words = stopwords.words('english')
stop_words.remove('not')

In [186]:
def remove_stopwords(text):
    hasil = ' '.join([kata for kata in text.split() if kata not in stop_words])
    return hasil 

### 6. Remove whitespace

In [187]:
def remove_whitespace(text):
    hasil = ' '.join(text.split())
    return hasil

### Combine all steps

In [188]:
df['Metadata'] = df['Metadata'].apply(to_lower)
df['Metadata'] = df['Metadata'].apply(remove_contraction)
df['Metadata'] = df['Metadata'].apply(remove_number)
df['Metadata'] = df['Metadata'].apply(remove_punctuation)
df['Metadata'] = df['Metadata'].apply(remove_stopwords)
df['Metadata'] = df['Metadata'].apply(remove_whitespace)

df.head()

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Metadata
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",drama two imprisoned men bond number years fin...
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",crime drama organized crime dynastys aging pat...
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",action crime drama menace known joker wreaks h...
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",crime drama early life career vito corleone ne...
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000",crime drama jury holdout attempts prevent misc...


## Preprocessing Part 2

Mengubah kata menjadi kata dasar.

Pilihan:
- Lemmatization

In [189]:
from nltk.stem import WordNetLemmatizer 

## Lemmatization

In [190]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\SWIFT\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [191]:
lemmatizer = WordNetLemmatizer()

In [192]:
def lemmatize(text):

    list_hasil = []

    for sentence in nltk.sent_tokenize(text):   # pecah paragraf menjadi kalimat-kalimat
        for word in nltk.word_tokenize(sentence): # pecah kalimat menjadi kata-kata
            list_hasil.append(lemmatizer.lemmatize(word))   # ubah kata ke kata dasarnya

    hasil = ' '.join(list_hasil)
    return hasil 

---
## Combine Preprocessing Part 1 & Part 2

In [193]:
df['lemmatize'] = df['Metadata'].apply(lemmatize)

df

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Metadata,lemmatize
0,https://m.media-amazon.com/images/M/MV5BMDFkYT...,The Shawshank Redemption,1994,A,142 min,Drama,9.3,Two imprisoned men bond over a number of years...,80.0,Frank Darabont,Tim Robbins,Morgan Freeman,Bob Gunton,William Sadler,2343110,"28,341,469",drama two imprisoned men bond number years fin...,drama two imprisoned men bond number year find...
1,https://m.media-amazon.com/images/M/MV5BM2MyNj...,The Godfather,1972,A,175 min,"Crime, Drama",9.2,An organized crime dynasty's aging patriarch t...,100.0,Francis Ford Coppola,Marlon Brando,Al Pacino,James Caan,Diane Keaton,1620367,"134,966,411",crime drama organized crime dynastys aging pat...,crime drama organized crime dynasty aging patr...
2,https://m.media-amazon.com/images/M/MV5BMTMxNT...,The Dark Knight,2008,UA,152 min,"Action, Crime, Drama",9.0,When the menace known as the Joker wreaks havo...,84.0,Christopher Nolan,Christian Bale,Heath Ledger,Aaron Eckhart,Michael Caine,2303232,"534,858,444",action crime drama menace known joker wreaks h...,action crime drama menace known joker wreaks h...
3,https://m.media-amazon.com/images/M/MV5BMWMwMG...,The Godfather: Part II,1974,A,202 min,"Crime, Drama",9.0,The early life and career of Vito Corleone in ...,90.0,Francis Ford Coppola,Al Pacino,Robert De Niro,Robert Duvall,Diane Keaton,1129952,"57,300,000",crime drama early life career vito corleone ne...,crime drama early life career vito corleone ne...
4,https://m.media-amazon.com/images/M/MV5BMWU4N2...,12 Angry Men,1957,U,96 min,"Crime, Drama",9.0,A jury holdout attempts to prevent a miscarria...,96.0,Sidney Lumet,Henry Fonda,Lee J. Cobb,Martin Balsam,John Fiedler,689845,"4,360,000",crime drama jury holdout attempts prevent misc...,crime drama jury holdout attempt prevent misca...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,https://m.media-amazon.com/images/M/MV5BNGEwMT...,Breakfast at Tiffany's,1961,A,115 min,"Comedy, Drama, Romance",7.6,A young New York socialite becomes interested ...,76.0,Blake Edwards,Audrey Hepburn,George Peppard,Patricia Neal,Buddy Ebsen,166544,NaN,comedy drama romance young new york socialite ...,comedy drama romance young new york socialite ...
996,https://m.media-amazon.com/images/M/MV5BODk3Yj...,Giant,1956,G,201 min,"Drama, Western",7.6,Sprawling epic covering the life of a Texas ca...,84.0,George Stevens,Elizabeth Taylor,Rock Hudson,James Dean,Carroll Baker,34075,NaN,drama western sprawling epic covering life tex...,drama western sprawling epic covering life tex...
997,https://m.media-amazon.com/images/M/MV5BM2U3Yz...,From Here to Eternity,1953,Passed,118 min,"Drama, Romance, War",7.6,"In Hawaii in 1941, a private is cruelly punish...",85.0,Fred Zinnemann,Burt Lancaster,Montgomery Clift,Deborah Kerr,Donna Reed,43374,"30,500,000",drama romance war hawaii private cruelly punis...,drama romance war hawaii private cruelly punis...
998,https://m.media-amazon.com/images/M/MV5BZTBmMj...,Lifeboat,1944,NaN,97 min,"Drama, War",7.6,Several survivors of a torpedoed merchant ship...,78.0,Alfred Hitchcock,Tallulah Bankhead,John Hodiak,Walter Slezak,William Bendix,26471,NaN,drama war several survivors torpedoed merchant...,drama war several survivor torpedoed merchant ...


## Encode Overview

In [194]:
tf = CountVectorizer(stop_words='english', tokenizer=word_tokenize) 
bow = tf.fit_transform(df['lemmatize']) 

In [195]:
idx = 0

content = df.loc[idx, 'lemmatize']
watched = tf.transform([content])
dist = cosine_distances(watched, bow)
rec_idx = dist.argsort()[0, 1:11]
df.loc[rec_idx]

,Poster_Link,Series_Title,Released_Year,Certificate,Runtime,Genre,IMDB_Rating,Overview,Meta_score,Director,Star1,Star2,Star3,Star4,No_of_Votes,Gross,Metadata,lemmatize
505,https://m.media-amazon.com/images/M/MV5BMTIzND...,Mystic River,2003,A,138 min,"Crime, Drama, Mystery",7.9,The lives of three men who were childhood frie...,84.0,Clint Eastwood,Sean Penn,Tim Robbins,Kevin Bacon,Emmy Rossum,419420,"90,135,191",crime drama mystery lives three men childhood ...,crime drama mystery life three men childhood f...
817,https://m.media-amazon.com/images/M/MV5BZWY0OD...,Short Cuts,1993,R,188 min,"Comedy, Drama",7.7,The day-to-day lives of several suburban Los A...,79.0,Robert Altman,Andie MacDowell,Julianne Moore,Tim Robbins,Bruce Davison,42275,"6,110,979",comedy drama daytoday lives several suburban l...,comedy drama daytoday life several suburban lo...
167,https://m.media-amazon.com/images/M/MV5BODM3YW...,Unforgiven,1992,A,130 min,"Drama, Western",8.2,Retired Old West gunslinger William Munny relu...,85.0,Clint Eastwood,Clint Eastwood,Gene Hackman,Morgan Freeman,Richard Harris,375935,"101,157,447",drama western retired old west gunslinger will...,drama western retired old west gunslinger will...
68,https://m.media-amazon.com/images/M/MV5BMTI3NT...,Oldeuboi,2003,A,101 min,"Action, Drama, Mystery",8.4,After being kidnapped and imprisoned for fifte...,77.0,Chan-wook Park,Choi Min-sik,Yoo Ji-Tae,Kang Hye-jeong,Kim Byeong-Ok,515451,"707,481",action drama mystery kidnapped imprisoned fift...,action drama mystery kidnapped imprisoned fift...
234,https://m.media-amazon.com/images/M/MV5BMTkxNz...,Million Dollar Baby,2004,UA,132 min,"Drama, Sport",8.1,A determined woman works with a hardened boxin...,86.0,Clint Eastwood,Hilary Swank,Clint Eastwood,Morgan Freeman,Jay Baruchel,635975,"100,492,203",drama sport determined woman works hardened bo...,drama sport determined woman work hardened box...
877,https://m.media-amazon.com/images/M/MV5BODQ0M2...,Dark Waters,2019,PG-13,126 min,"Biography, Drama, History",7.6,A corporate defense attorney takes on an envir...,73.0,Todd Haynes,Mark Ruffalo,Anne Hathaway,Tim Robbins,Bill Pullman,60408,NaN,biography drama history corporate defense atto...,biography drama history corporate defense atto...
311,https://m.media-amazon.com/images/M/MV5BNzc1MT...,The Shop Around the Corner,1940,NaN,99 min,"Comedy, Drama, Romance",8.1,Two employees at a gift shop can barely stand ...,96.0,Ernst Lubitsch,Margaret Sullavan,James Stewart,Frank Morgan,Joseph Schildkraut,28450,"203,300",comedy drama romance two employees gift shop b...,comedy drama romance two employee gift shop ba...
27,https://m.media-amazon.com/images/M/MV5BOTUwOD...,Se7en,1995,A,127 min,"Crime, Drama, Mystery",8.6,"Two detectives, a rookie and a veteran, hunt a...",65.0,David Fincher,Morgan Freeman,Brad Pitt,Kevin Spacey,Andrew Kevin Walker,1445096,"100,125,643",crime drama mystery two detectives rookie vete...,crime drama mystery two detective rookie veter...
350,https://m.media-amazon.com/images/M/MV5BYjU1Nj...,Dev.D,2009,A,144 min,"Drama, Romance",8.0,After breaking up with his childhood sweethear...,NaN,Anurag Kashyap,Abhay Deol,Mahie Gill,Kalki Koechlin,Dibyendu Bhattacharya,28749,"10,950",drama romance breaking childhood sweetheart yo...,drama romance breaking childhood sweetheart yo...
922,https://m.media-amazon.com/images/M/MV5BYmM2ND...,Gone Baby Gone,2007,R,114 min,"Crime, Drama, Mystery",7.6,Two Boston area detectives investigate a littl...,72.0,Ben Affleck,Morgan Freeman,Ed Harris,Casey Affleck,Michelle Monaghan,250590,"20,300,218",crime drama mystery two boston area detectives...,crime drama mystery two boston area detective ...


In [197]:
df.to_csv('dataset/movie_clean.csv',index=False)

In [198]:
import pickle

pickle.dump(tf,open('model/tf.pkl','wb'))
pickle.dump(bow,open('model/bow.pkl','wb'))